<a href="https://colab.research.google.com/github/ANGELLOPARR/csc466-project/blob/main/NLP_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
import nltk
import spacy
nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, Input, Model
import tqdm
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix

!python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [3]:
!rm -r csc466-project
!git clone https://github.com/ANGELLOPARR/csc466-project.git

rm: cannot remove 'csc466-project': No such file or directory
Cloning into 'csc466-project'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 83 (delta 34), reused 25 (delta 4), pack-reused 0
Unpacking objects: 100% (83/83), done.


In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

dataset = pd.read_csv(r'csc466-project/emails.csv')
dataset .columns #Index(['text', 'spam'], dtype='object')
dataset.shape  #(5728, 2)
dataset.iloc[-1]['text']

'Subject: news : aurora 5 . 2 update  aurora version 5 . 2  - the fastest model just got faster -  epis announces the release of aurora , version 5 . 2  aurora the electric market price forecasting tool is already  legendary for power and speed . we \' ve combined a powerful chronological  dispatch model with the capability to simulate the market from 1  day to 25 + years . add to that a risk analysis section , powered by user  selectable monte carlo & / or latin hypercube modeling , enough  portfolio analysis power to please the toughest critic , & inputs and  outputs from standard excel & access tables and you \' ve got one of most  powerful tools in the market .  just a few months ago we expanded our emissions modeling  capabilities , added our quarterly database update , increased the speed  of the entire model , and made  but that wasn \' t enough .  we \' ve done it again . some of the operations that we \' ve  included . . .  two new reporting enhancements .  the first is margin

Data cleanup

In [5]:
dataset['text'] = dataset['text'].str.replace('Subject: ', '')
dataset

,text,spam
0,naturally irresistible your corporate identity...,1
1,the stock trading gunslinger fanny is merrill...,1
2,unbelievable new homes made easy im wanting t...,1
3,4 color printing special request additional i...,1
4,"do not have money , get software cds from here...",1
...,...,...
5723,re : research and development charges to gpg ...,0
5724,"re : receipts from visit jim , thanks again ...",0
5725,re : enron case study update wow ! all on the...,0
5726,"re : interest david , please , call shirley ...",0


In [6]:
def stem_text(text):
    stemmer = PorterStemmer()
    tokenized = nltk.word_tokenize(text)
    stemmed = [stemmer.stem(word) for word in tokenized]
    return ' '.join(stemmed)

In [7]:
stem_text(dataset.iloc[0]['text'])

"natur irresist your corpor ident lt is realli hard to recollect a compani : the market is full of suqgest and the inform isoverwhelminq ; but a good catchi logo , stylish statloneri and outstand websit will make the task much easier . we do not promis that havinq order a iogo your compani will automaticaili becom a world ieader : it isguit ciear that without good product , effect busi organ and practic aim it will be hotat nowaday market ; but we do promis that your market effort will becom much more effect . here is the list of clear benefit : creativ : hand - made , origin logo , special done to reflect your distinct compani imag . conveni : logo and stationeri are provid in all format ; easi - to - use content manag system letsyou chang your websit content and even it structur . prompt : you will see logo draft within three busi day . afford : your market break - through shouldn ' t make gap in your budget . 100 % satisfact guarante : we provid unlimit amount of chang with no extra

In [8]:
def lemmatize_text(text):
  doc = nlp(text)

  lemmed = [token.lemma_ if token.lemma_ != '-PRON-' else token.text for token in doc]
  return ' '.join(lemmed)

In [9]:
lemmatize_text(dataset.iloc[0]['text'])

"naturally irresistible your corporate identity   lt be really hard to recollect a company : the   market be full of suqgestion and the information isoverwhelminq ; but a good   catchy logo , stylish statlonery and outstanding website   will make the task much easy .   we do not promise that havinq order a iogo your   company will automaticaily become a world ieader : it isguite ciear that   without good product , effective business organization and practicable aim it   will be hotat nowadays market ; but we do promise that your marketing effort   will become much more effective . here be the list of clear   benefit : creativeness : hand - make , original logo , specially do   to reflect your distinctive company image . convenience : logo and stationery   be provide in all format ; easy - to - use content management system letsyou   change your website content and even its structure . promptness : you   will see logo draft within three business day . affordability : your   marketing br

In [13]:
corpus = dataset['text'].values
stemmed_corpus = [stem_text(document) for document in tqdm(corpus)]
lemmed_corpus = [lemmatize_text(document) for document in tqdm(corpus)]

100%|██████████| 5728/5728 [09:49<00:00,  9.71it/s]


In [27]:
stemmed_df = dataset.copy(deep=True)
stemmed_df['stemmed'] = stemmed_corpus
stemmed_df.drop(['text'], axis=1, inplace=True)

X = stemmed_df['stemmed']
t = stemmed_df['spam']

X_train_stemmed, X_test_stemmed, t_train_stemmed, t_test_stemmed = train_test_split(X, t, test_size=0.2, random_state=0)

In [37]:
lemmed_df = dataset.copy(deep=True)
lemmed_df['lemmed'] = lemmed_corpus
lemmed_df.drop(['text'], axis=1, inplace=True)

X = lemmed_df['lemmed']
t = lemmed_df['spam']

X_train_lemmed, X_test_lemmed, t_train_lemmed, t_test_lemmed = train_test_split(X, t, test_size=0.2, random_state=0)

In [34]:
stemmed_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')

stemmed_vectorizer.fit(X_train_stemmed)

X2_train_stemmed = stemmed_vectorizer.transform(X_train_stemmed)
X2_test_stemmed = stemmed_vectorizer.transform(X_test_stemmed)
X2_train_stemmed = pd.DataFrame(X2_train_stemmed.todense(),
                                columns=stemmed_vectorizer.get_feature_names(),
                                index=X_train_stemmed.index)
X2_test_stemmed = pd.DataFrame(X2_test_stemmed.todense(),
                               columns=stemmed_vectorizer.get_feature_names(),
                               index=X_test_stemmed.index)
X2_test_stemmed

,00,000,0000,001,0011,0044,01,0100,011,02,020,0200,03,04,0400,0413,05,0500,0596,06,0600,07,070700,08,09,0985,10,100,1000,100038,101,105,1050,107043,1092,11,110,115,1179,1188,...,xara,xia,xiao,xl,xll,xm,xp,xpress,xxx,yahoo,yaman,yan,yana,yanni,ye,yeager,year,yen,yeow,yesterday,yho,yield,york,young,youngblood,youyi,yr,yuan,za,zadorozhni,ze,zero,zhang,zhendong,zimbabw,zimin,zip,ziplip,zipter,zone
4595,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.030937,0.0,0.0,0.0,0.0,0.0,0.0,0.016961,0.0,0.017164,0.0,0.000000,0.000000,0.0,0.012179,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.029725,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.043530,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.030526,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
1744,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
29,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.093135,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
98,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
539,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.07113,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3358,0.0,0.0,0.0,0.0,0.0,0.0,0.020786,0.0,0.0,0.068669,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.100335,0.0,0.000000,0.0,0.023453,0.023481,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.042041,0.0,0.0,0.0,0.00000,0.0,0.042918,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050054,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.045677,0.0
242,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.075874,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
3037,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.145650,0.0,0.0,0.0,0.04975

In [35]:
t_test_stemmed

4595    0
1744    0
29      1
98      1
539     1
       ..
3358    0
242     1
3037    0
3683    0
651     1
Name: spam, Length: 1146, dtype: int64

In [40]:
lemmed_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')

lemmed_vectorizer.fit(X_train_lemmed)

X2_train_lemmed = lemmed_vectorizer.transform(X_train_lemmed)
X2_test_lemmed = lemmed_vectorizer.transform(X_test_lemmed)
X2_train_lemmed = pd.DataFrame(X2_train_lemmed.todense(),
                                columns=lemmed_vectorizer.get_feature_names(),
                                index=X_train_lemmed.index)
X2_test_lemmed = pd.DataFrame(X2_test_lemmed.todense(),
                               columns=lemmed_vectorizer.get_feature_names(),
                               index=X_test_lemmed.index)

In [45]:
X2_test_lemmed

,00,000,0000,001,0011,01,011,02,020,0200,03,04,0400,05,0500,0596,06,0600,07,070700,08,09,0985,10,100,1000,100038,101,105,1050,107043,1092,11,110,115,1179,1188,119,12,120,...,wscc,wti,www,xara,xia,xiao,xll,xls,xms,xp,xpress,xxx,yahoo,yaman,yana,yannis,yeager,year,yen,yeow,yes,yesterday,yho,yield,york,young,youyi,yr,yuan,za,zadorozhny,ze,zero,zhang,zhendong,zimin,zip,ziplip,zipter,zone
4595,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.031443,0.0,0.0,0.0,0.0,0.0,0.017238,0.0,0.017445,0.0,0.000000,0.000000,0.0,0.012378,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03021,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.044241,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.031024,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
1744,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
29,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.092985,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
98,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
539,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.073844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3358,0.0,0.0,0.0,0.0,0.0,0.020834,0.0,0.068828,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.100567,0.0,0.000000,0.0,0.023508,0.023536,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041806,0.0,0.0,0.0,0.043017,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05017,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.045783,0.0
242,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.075442,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
3037,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.

In [79]:
bs = int(len(X2_train_stemmed) * 0.02) # 4% of input samples per batch
nepochs = 4

In [80]:
inputs = Input(shape=(X2_train_stemmed.shape[1],))

nn = layers.Dense(1024, activation='relu')(inputs)
nn = layers.Dense(512, activation='relu')(nn)
nn = layers.Dense(256, activation='relu')(nn)
nn = layers.Dense(128, activation='relu')(nn)
nn = layers.Dense(64, activation='relu')(nn)
nn = layers.Dense(32, activation='relu')(nn)

outputs = layers.Dense(1, activation='sigmoid')(nn)
model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 5000)]            0         
_________________________________________________________________
dense_39 (Dense)             (None, 1024)              5121024   
_________________________________________________________________
dense_40 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_41 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_42 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_43 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_44 (Dense)             (None, 32)                2080

In [81]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X2_train_stemmed,
                    t_train_stemmed,
                    batch_size=bs,
                    epochs=nepochs,
                    validation_split=0.10)
model.evaluate(X2_test_stemmed, t_test_stemmed)
t_pred = model.predict(X2_test_stemmed)

Epoch 1/4
46/46 [==============================] - 17s 367ms/step - loss: 0.3181 - accuracy: 0.8326 - val_loss: 0.0424 - val_accuracy: 0.9847
Epoch 2/4
46/46 [==============================] - 3s 73ms/step - loss: 0.0161 - accuracy: 0.9964 - val_loss: 0.0218 - val_accuracy: 0.9935
Epoch 3/4
46/46 [==============================] - 3s 72ms/step - loss: 0.0021 - accuracy: 0.9995 - val_loss: 0.0387 - val_accuracy: 0.9891
Epoch 4/4
36/36 [==============================] - 1s 15ms/step - loss: 0.0466 - accuracy: 0.9895


In [83]:
inputs = Input(shape=(X2_train_stemmed.shape[1],))

nn = layers.Dense(1024, activation='relu')(inputs)
nn = layers.Dense(512, activation='relu')(nn)
nn = layers.Dense(256, activation='relu')(nn)
nn = layers.Dense(128, activation='relu')(nn)
nn = layers.Dense(64, activation='relu')(nn)
nn = layers.Dense(32, activation='relu')(nn)

outputs = layers.Dense(1, activation='sigmoid')(nn)
model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 5000)]            0         
_________________________________________________________________
dense_46 (Dense)             (None, 1024)              5121024   
_________________________________________________________________
dense_47 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_48 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_49 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_50 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_51 (Dense)             (None, 32)                2080

In [84]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X2_train_lemmed,
                    t_train_lemmed,
                    batch_size=bs,
                    epochs=nepochs,
                    validation_split=0.10)
model.evaluate(X2_test_lemmed, t_test_lemmed)
t_pred = model.predict(X2_test_lemmed)

Epoch 1/4
46/46 [==============================] - 4s 79ms/step - loss: 0.2485 - accuracy: 0.8785 - val_loss: 0.0362 - val_accuracy: 0.9891
Epoch 2/4
46/46 [==============================] - 3s 74ms/step - loss: 0.0106 - accuracy: 0.9983 - val_loss: 0.0673 - val_accuracy: 0.9826
Epoch 3/4
46/46 [==============================] - 3s 72ms/step - loss: 0.0027 - accuracy: 0.9993 - val_loss: 0.0403 - val_accuracy: 0.9913
Epoch 4/4
36/36 [==============================] - 1s 14ms/step - loss: 0.0363 - accuracy: 0.9921
